In [ ]:
import sys 
import sqlite3
import argparse
import os 
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
load_dotenv()

In [ ]:
# We need to create a connection to the database file, named vocab.db
# We have retrieved this file from our kindle, and it contains the words we have looked up
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(e)
    return conn

In [ ]:
# We want to joint the two tables from LOOKUPS, BOOK_INFO and WORDS
# We can do this by making use of the book_key from LOOKUPS and id from BOOK_INFO
# To join the other tables we have to make use of the word_key from LOOKUPS and id from WORDS
# The only columns we need are: word_key/id, word, title, usage
def join_tables(conn):
    cur = conn.cursor()
    cur.execute("SELECT word_key, word, title, usage FROM LOOKUPS JOIN WORDS ON LOOKUPS.word_key = WORDS.id JOIN BOOK_INFO ON LOOKUPS.book_key = BOOK_INFO.id")
    rows = cur.fetchall()
    return rows

In [ ]:
# Let's try our code
def retrieve_rows(file):
    conn = create_connection(file)
    with conn:
        rows = join_tables(conn)
        return rows

In [ ]:
file = 'vocab.db'
rows = retrieve_rows(file)

In [ ]:
# We want to create a dataframe from the rows we have retrieved, so we can filter on a book title
# Change the column names to something more readable: Word, Book, Context
# We don't need the word_key, so we can drop this column
def create_dataframe(rows):
    df = pd.DataFrame(rows, columns=['Word_key', 'Word', 'Book', 'Context'])
    df = df.drop(columns=['Word_key'])
    return df

In [ ]:
# Create an OpenAI client to use the API
client = OpenAI()

In [ ]:
# For each word in our joined tables, we want to determine the meaning of the word in the context of the usage. 
# We will use the OpenAI client to generate a response for each word
def generate_definition(word, usage):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a language bot who EXCLUSIVELY returns the English definition of the word given the context, without explanation. The translation must at most be several words."},
            {
                "role": "user",
                "content": f"Define \"{word}\" in the context of \"{usage}\"."
            }
        ]
    )
    return completion.choices[0].message.content


In [ ]:
df = create_dataframe(rows)

In [ ]:
# Function to get all the books from the dataframe
def get_books(df):
    return df['Book'].unique()

# Print the books in the dataframe and index them
def print_books(books):
    for index, book in enumerate(books):
        print(f"{index}: {book}")

# Let a user choose a book from the list of books, by providing the index of the book
# We will then filter the dataframe on the chosen book
def choose_book(df):
    books = get_books(df)
    print_books(books)
    book_index = int(input("Choose a book by providing the index: "))
    book = books[book_index]
    return df[df['Book'] == book]


In [ ]:
# Filter the dataframe on the chosen book
filtered_words = choose_book(df)

In [ ]:
# Take first 10 rows for testing
filtered_words_ten = filtered_words.head(10)

# For each row in the filtered words, we want to generate a definition and directly add this to the dataframe
definitions = []
results = []
for index, row in filtered_words_ten.iterrows():
    word = row[0]
    usage = row[2]
    definition = generate_definition(word, usage)
    definitions.append(definition)
    #a.append_row([row[1], response, row[3]])
    results.append([row[0], definition, row[2], row[1]])
df = pd.DataFrame(results, columns=['Word', 'Definition', 'Context', 'Book'])



In [ ]:
df

In [ ]:
# Save the dataframe to a csv file
df.to_csv('definitions.csv', index=False)